In [ ]:
import pandas as pd
import os
import openai
import requests
import re
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:
unsuccessful=[]
sentences=[]
delimiter = "####"
# GitHub folder URL
github_folder_url = 'https://github.com/atreish/yukatzbot/tree/main/Text%20data'

# Fetch the list of text files from the GitHub folder
response = requests.get(github_folder_url)
response.raise_for_status()
folder_contents = response.text

# Extract the filenames
file_names = re.findall(r'(?<=href="/atreish/yukatzbot/blob/main/Text%20data/).*?\.txt', folder_contents)

# Process each text file
for file_name in file_names:
    # Download the text file content
    raw_file_url = f'https://raw.githubusercontent.com/atreish/yukatzbot/main/Text%20data/{file_name}'
    response = requests.get(raw_file_url)
    response.raise_for_status()
    text_content = response.text

    user_message_1 = f"""
    What are some 10 questions that can be answered with the above text in system message? What are corresponding 10 answers for these questions? """
    messages =  [
{'role':'system',
 'content': text_content},
{'role':'user',
 'content': f"{delimiter}{user_message_1}{delimiter}"},
]
    try:
        category_and_product_respons = get_completion_from_messages(messages)
        sentences.extend(category_and_product_respons.split('\n'))

    except openai.OpenAIError as e:
        # Handle the OpenAIError exception
        unsuccessful.append(text_content)
        # Additional error handling code if needed

In [ ]:
len(sentences)

In [ ]:
sentences=list(filter(lambda x: x!='I can provide five questions and their corresponding answers based on the above text:',sentences))

In [ ]:
sentences=list(filter(lambda x: x!="I'm sorry, as an AI language model, I cannot provide a list of 10 questions and their corresponding answers without any context. Could you please provide more information or specify the questions you would like me to answer?",sentences))

In [ ]:
sentences=list(filter(lambda x: x!='',sentences))

<ipython-input-53-dafbd3c9932e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['question']=df2['question'].apply(lambda x: re.sub(r'(Question)*\s+\d+:','',x))


In [ ]:
df=pd.DataFrame(columns=['question','answer'])

<ipython-input-61-3022c8a19c3d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['question']=df2['question'].apply(lambda x: re.sub(r'Question','',x))


In [ ]:
for i in range(0,len(sentences),2) :
    df.loc[len(df)]={
        'question':sentences[i],
          'answer':sentences[i+1],

    }

<ipython-input-54-e3d18d911fb2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['answer']=df2['answer'].apply(lambda x: re.sub(r'(Answer)*\s+\d+:','',x))


In [ ]:
df['answer']=df['answer'].apply(lambda x: re.sub(r'Answer: ','',x))

In [ ]:
df['question']=df['question'].apply(lambda x: re.sub(r'\d+\.\s','',x))

In [ ]:
df['answer']=df['answer'].apply(lambda x: re.sub(r'^\d+\.\s*','',x))

<ipython-input-69-fea8bc8f5d90>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['answer']=df2['answer'].apply(lambda x: re.sub(r'\d+.','',x))


In [ ]:
df2['question']=df2['question'].apply(lambda x: re.sub(r'Q:','',x))

<ipython-input-55-f1dc7e79338b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['question']=df2['question'].apply(lambda x: re.sub(r'Q:','',x))


In [ ]:
df2['answer']=df2['answer'].apply(lambda x: re.sub(r'Answer:','',x))

<ipython-input-70-d69721692162>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['answer']=df2['answer'].apply(lambda x: re.sub(r'Answer:','',x))


In [ ]:
df.to_csv('questions_answers_cleaned.csv', index=False)